# Training with EgoVLP Backbone

Train MLP, Transformer, and RNN models with EgoVLP video backbone.

## Workflow:
1. Mount Google Drive
2. Clone repository and install dependencies
3. **Load annotations from Google Drive**
4. **Load pre-extracted EgoVLP features from Google Drive**
5. Train models (MLP, Transformer, RNN) with EgoVLP backbone
6. Evaluate and compare results

## Prerequisites:
- Set Colab secrets: `WANDB_API_KEY`
- **Upload EgoVLP features zip file** (`egovlp.zip`) to Google Drive at `MyDrive/AML_mistake_detection/features/`
- **Upload CaptainCook4D annotations to Google Drive** at `MyDrive/AML_mistake_detection/annotations/`


## 1. Configuration


In [ ]:
# Configuration
HF_DATASET_REPO = "aexomir/captaincook4d-features" 
REPO_URL = "https://github.com/aexomir/AML_mistake_detection.git"
REPO_BRANCH = "feat/extend-baseline"

# Google Drive path for annotations
ANNOTATIONS_DRIVE_PATH = '/content/drive/MyDrive/AML_mistake_detection/annotations'

# Backbone to train (EgoVLP only)
BACKBONE = "egovlp"
# Model variants
VARIANTS = ["MLP", "Transformer", "RNN"]

print(f"Will train {len(VARIANTS)} variants with {BACKBONE} backbone")
print(f"Backbone: {BACKBONE}")
print(f"Variants: {VARIANTS}")


## 2. Mount Google Drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


## 3. Clone Repository and Install Dependencies


In [ ]:
%%bash -s "$REPO_URL" "$REPO_BRANCH"
cd /content
if [ ! -d "AML_mistake_detection" ]; then
    git clone --branch "$2" "$1" AML_mistake_detection
else
    cd AML_mistake_detection && git pull
fi


In [ ]:
import os
os.chdir('/content/AML_mistake_detection')

%pip install -q -r requirements.txt
%pip install -q wandb huggingface_hub

print("✓ Dependencies installed")


## 4. Load Annotations from Google Drive


In [ ]:
import os
import shutil

print("Loading CaptainCook4D annotations...")

if os.path.exists(ANNOTATIONS_DRIVE_PATH):
    print(f"✓ Found annotations at: {ANNOTATIONS_DRIVE_PATH}")
    
    # Create destination directories if they don't exist
    os.makedirs('annotations/annotation_json', exist_ok=True)
    os.makedirs('annotations/data_splits', exist_ok=True)
    
    # Copy annotation_json (step_annotations.json, error_annotations.json)
    annotation_json_src = os.path.join(ANNOTATIONS_DRIVE_PATH, 'annotation_json')
    if os.path.exists(annotation_json_src):
        print("  Copying annotation_json files...")
        for file in os.listdir(annotation_json_src):
            src = os.path.join(annotation_json_src, file)
            dst = os.path.join('annotations/annotation_json', file)
            if os.path.isfile(src):
                shutil.copy2(src, dst)
                print(f"    ✓ Copied {file}")
    else:
        print(f"  ⚠️  annotation_json not found at {annotation_json_src}")
    
    # Copy data_splits (optional)
    data_splits_src = os.path.join(ANNOTATIONS_DRIVE_PATH, 'data_splits')
    if os.path.exists(data_splits_src):
        print("  Copying data_splits files...")
        for file in os.listdir(data_splits_src):
            src = os.path.join(data_splits_src, file)
            dst = os.path.join('annotations/data_splits', file)
            if os.path.isfile(src):
                shutil.copy2(src, dst)
                print(f"    ✓ Copied {file}")
    
    # Verify required files
    required_files = [
        'annotations/annotation_json/step_annotations.json',
        'annotations/annotation_json/error_annotations.json',
        'er_annotations/recordings_combined_splits.json'
    ]
    
    missing_files = [f for f in required_files if not os.path.exists(f)]
    
    if missing_files:
        print(f"\n❌ ERROR: Missing required annotation files:")
        for file in missing_files:
            print(f"  - {file}")
        print("\nPlease ensure these files are in your Google Drive at:")
        print(f"  {ANNOTATIONS_DRIVE_PATH}/")
        raise FileNotFoundError("Missing required annotation files")
    else:
        print("\n✅ All required annotation files loaded successfully!")
        
else:
    print(f"❌ ERROR: Annotations directory not found at: {ANNOTATIONS_DRIVE_PATH}")
    print("\nPlease upload CaptainCook4D annotations to Google Drive:")
    print("  Required structure:")
    print("  MyDrive/AML_mistake_detection/annotations/")
    print("    ├── annotation_json/")
    print("    │   ├── step_annotations.json")
    print("    │   └── error_annotations.json")
    print("    └── data_splits/ (optional)")
    print("\nDownload from: https://captaincook4d.github.io/captain-cook/")
    raise FileNotFoundError("Annotations directory not found in Google Drive")


## 5. Load EgoVLP Features from Google Drive


In [ ]:
import zipfile
import os
from pathlib import Path

# Google Drive path for features zip file
FEATURES_DRIVE_PATH = '/content/drive/MyDrive/AML_mistake_detection/features'
ZIP_FILE_NAME = f'{BACKBONE}.zip'
zip_file_path = os.path.join(FEATURES_DRIVE_PATH, ZIP_FILE_NAME)

# Create data/features directory structure
features_base = Path('/content/AML_mistake_detection/data/features')
backbone_dir = features_base / BACKBONE
backbone_dir.mkdir(parents=True, exist_ok=True)

print(f"Loading {BACKBONE} features from Google Drive...")

# Check if zip file exists
if not os.path.exists(zip_file_path):
    print(f"❌ ERROR: Zip file not found at: {zip_file_path}")
    print(f"\nPlease ensure {ZIP_FILE_NAME} is uploaded to Google Drive at:")
    print(f"  {FEATURES_DRIVE_PATH}/")
    raise FileNotFoundError(f"Features zip file not found: {zip_file_path}")

print(f"✓ Found zip file: {zip_file_path}")

# Extract zip file if directory is empty or doesn't exist
if not list(backbone_dir.glob('*.npy')):
    print(f"Extracting {BACKBONE} features...")
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        # Extract all files
        zip_ref.extractall(backbone_dir)
        # If files were extracted with subdirectory structure, move them to root
        for root, dirs, files in os.walk(backbone_dir):
            for file in files:
                if file.endswith('.npy'):
                    src = Path(root) / file
                    dst = backbone_dir / file
                    if src != dst and not dst.exists():
                        src.rename(dst)
        # Clean up empty subdirectories
        for root, dirs, files in os.walk(backbone_dir, topdown=False):
            for dir_name in dirs:
                dir_path = Path(root) / dir_name
                try:
                    if not any(dir_path.iterdir()):
                        dir_path.rmdir()
                except OSError:
                    pass
    print(f"✓ Extracted {BACKBONE} features")
else:
    print(f"✓ Features already extracted (found {len(list(backbone_dir.glob('*.npy')))} files)")

print(f"✓ Total files: {len(list(backbone_dir.glob('*.npy')))}")


## 6. Setup WandB


In [ ]:
import wandb

# Login to WandB
wandb_key = userdata.get('WANDB_API_KEY')
wandb.login(key=wandb_key)

print("✓ WandB configured")


## 7. Train Models


In [ ]:
import subprocess
import sys

# Training configuration
SPLIT = "recordings"
THRESHOLD = 0.6
EPOCHS = 100
BATCH_SIZE = 128

training_results = {}

for variant in VARIANTS:
    model_name = f"{variant}_{BACKBONE}"
    print(f"\n{'='*60}")
    print(f"Training: {model_name}")
    print(f"{'='*60}")
    
    # Select appropriate training script
    if variant == "RNN":
        train_script = "scripts/train_rnn_baseline.py"
    else:
        train_script = "train_er.py"
    
    # Build training command
    cmd = [
        sys.executable, train_script,
        "--backbone", BACKBONE,
        "--variant", variant,
        "--split", SPLIT,
        "--threshold", str(THRESHOLD),
        "--epochs", str(EPOCHS),
        "--batch_size", str(BATCH_SIZE),
        "--use_wandb"
    ]
    
    try:
        # Run training
        result = subprocess.run(cmd, check=True, capture_output=True, text=True)
        print(result.stdout)
        training_results[model_name] = "SUCCESS"
        print(f"✓ {model_name} training completed")
    except subprocess.CalledProcessError as e:
        print(f"✗ {model_name} training failed:")
        print(e.stderr)
        training_results[model_name] = "FAILED"

print(f"\n{'='*60}")
print("Training Summary:")
print(f"{'='*60}")
for model, status in training_results.items():
    status_icon = "✓" if status == "SUCCESS" else "✗"
    print(f"{status_icon} {model}: {status}")


## 8. Evaluate Models


In [ ]:
evaluation_results = {}

for variant in VARIANTS:
    model_name = f"{variant}_{BACKBONE}"
    print(f"\n{'='*60}")
    print(f"Evaluating: {model_name}")
    print(f"{'='*60}")
    
    # Build evaluation command
    cmd = [
        sys.executable, "core/evaluate.py",
        "--backbone", BACKBONE,
        "--variant", variant,
        "--split", SPLIT,
        "--threshold", str(THRESHOLD)
    ]
    
    try:
        # Run evaluation
        result = subprocess.run(cmd, check=True, capture_output=True, text=True)
        print(result.stdout)
        evaluation_results[model_name] = "SUCCESS"
        print(f"✓ {model_name} evaluation completed")
    except subprocess.CalledProcessError as e:
        print(f"✗ {model_name} evaluation failed:")
        print(e.stderr)
        evaluation_results[model_name] = "FAILED"

print(f"\n{'='*60}")
print("Evaluation Summary:")
print(f"{'='*60}")
for model, status in evaluation_results.items():
    status_icon = "✓" if status == "SUCCESS" else "✗"
    print(f"{status_icon} {model}: {status}")


## 9. Generate Comparison Report


In [ ]:
# Extract metrics from this notebook's outputs
print("Extracting metrics...")
cmd = [sys.executable, "analysis/extract_metrics.py"]
subprocess.run(cmd, check=True)

# Generate comparison tables
print("\nGenerating comparison tables...")
cmd = [sys.executable, "analysis/comparison_tables.py"]
subprocess.run(cmd, check=True)

# Generate visualizations
print("\nGenerating visualizations...")
cmd = [sys.executable, "analysis/comparison_visualizations.py"]
subprocess.run(cmd, check=True)

print("\n✓ All analysis complete!")
print("Check analysis/outputs/ for results")


# Training with New Backbones: EgoVLP & PerceptionEncoder

Train MLP, Transformer, and RNN models with new video backbones.

## Workflow:
1. Clone repository and install dependencies
2. **Load annotations from Google Drive**
3. **Load pre-extracted features from Google Drive**
4. Train models with EgoVLP and PerceptionEncoder backbones
5. Evaluate and compare results

## Prerequisites:
- Set Colab secrets: `WANDB_API_KEY`
- **Upload feature zip files** (e.g., `egovlp.zip`, `perceptionencoder.zip`) to Google Drive at `MyDrive/AML_mistake_detection/features/`
- **Upload CaptainCook4D annotations to Google Drive** at `MyDrive/AML_mistake_detection/annotations/`


## 1. Configuration


In [ ]:
# Configuration
HF_DATASET_REPO = "aexomir/captaincook4d-features" 
REPO_URL = "https://github.com/aexomir/AML_mistake_detection.git"
REPO_BRANCH = "feat/extend-baseline"

# Google Drive path for annotations
ANNOTATIONS_DRIVE_PATH = '/content/drive/MyDrive/AML_mistake_detection/annotations'

# Backbones to train
BACKBONES = ["egovlp"]
# Model variants
VARIANTS = ["MLP", "Transformer", "RNN"]

print(f"Will train {len(BACKBONES)} backbones × {len(VARIANTS)} variants = {len(BACKBONES) * len(VARIANTS)} models")
print(f"Backbones: {BACKBONES}")
print(f"Variants: {VARIANTS}")


## 2. Mount Google Drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


## 3. Clone Repository and Install Dependencies


In [ ]:
%%bash -s "$REPO_URL" "$REPO_BRANCH"
cd /content
if [ ! -d "AML_mistake_detection" ]; then
    git clone --branch "$2" "$1" AML_mistake_detection
else
    cd AML_mistake_detection && git pull
fi


In [ ]:
import os
os.chdir('/content/AML_mistake_detection')

%pip install -q -r requirements.txt
%pip install -q wandb huggingface_hub

print("✓ Dependencies installed")


## 4. Load Annotations from Google Drive


In [ ]:
import os
import shutil

print("Loading CaptainCook4D annotations...")

if os.path.exists(ANNOTATIONS_DRIVE_PATH):
    print(f"✓ Found annotations at: {ANNOTATIONS_DRIVE_PATH}")
    
    # Create destination directories if they don't exist
    os.makedirs('annotations/annotation_json', exist_ok=True)
    os.makedirs('annotations/data_splits', exist_ok=True)
    
    # Copy annotation_json (step_annotations.json, error_annotations.json)
    annotation_json_src = os.path.join(ANNOTATIONS_DRIVE_PATH, 'annotation_json')
    if os.path.exists(annotation_json_src):
        print("  Copying annotation_json files...")
        for file in os.listdir(annotation_json_src):
            src = os.path.join(annotation_json_src, file)
            dst = os.path.join('annotations/annotation_json', file)
            if os.path.isfile(src):
                shutil.copy2(src, dst)
                print(f"    ✓ Copied {file}")
    else:
        print(f"  ⚠️  annotation_json not found at {annotation_json_src}")
    
    # Copy data_splits (optional)
    data_splits_src = os.path.join(ANNOTATIONS_DRIVE_PATH, 'data_splits')
    if os.path.exists(data_splits_src):
        print("  Copying data_splits files...")
        for file in os.listdir(data_splits_src):
            src = os.path.join(data_splits_src, file)
            dst = os.path.join('annotations/data_splits', file)
            if os.path.isfile(src):
                shutil.copy2(src, dst)
                print(f"    ✓ Copied {file}")
    
    # Verify required files
    required_files = [
        'annotations/annotation_json/step_annotations.json',
        'annotations/annotation_json/error_annotations.json',
        'er_annotations/recordings_combined_splits.json'
    ]
    
    missing_files = [f for f in required_files if not os.path.exists(f)]
    
    if missing_files:
        print(f"\n❌ ERROR: Missing required annotation files:")
        for file in missing_files:
            print(f"  - {file}")
        print("\nPlease ensure these files are in your Google Drive at:")
        print(f"  {ANNOTATIONS_DRIVE_PATH}/")
        raise FileNotFoundError("Missing required annotation files")
    else:
        print("\n✅ All required annotation files loaded successfully!")
        
else:
    print(f"❌ ERROR: Annotations directory not found at: {ANNOTATIONS_DRIVE_PATH}")
    print("\nPlease upload CaptainCook4D annotations to Google Drive:")
    print("  Required structure:")
    print("  MyDrive/AML_mistake_detection/annotations/")
    print("    ├── annotation_json/")
    print("    │   ├── step_annotations.json")
    print("    │   └── error_annotations.json")
    print("    └── data_splits/ (optional)")
    print("\nDownload from: https://captaincook4d.github.io/captain-cook/")
    raise FileNotFoundError("Annotations directory not found in Google Drive")


## 5. Load Features from Google Drive


In [ ]:
import zipfile
import os
from pathlib import Path

# Google Drive path for features zip files
FEATURES_DRIVE_PATH = '/content/drive/MyDrive/AML_mistake_detection/features'

# Create data/features directory structure
features_base = Path('/content/AML_mistake_detection/data/features')

for backbone in BACKBONES:
    backbone_dir = features_base / backbone
    backbone_dir.mkdir(parents=True, exist_ok=True)
    
    zip_file_name = f'{backbone}.zip'
    zip_file_path = os.path.join(FEATURES_DRIVE_PATH, zip_file_name)
    
    print(f"Loading {backbone} features from Google Drive...")
    
    # Check if zip file exists
    if not os.path.exists(zip_file_path):
        print(f"❌ ERROR: Zip file not found at: {zip_file_path}")
        print(f"\nPlease ensure {zip_file_name} is uploaded to Google Drive at:")
        print(f"  {FEATURES_DRIVE_PATH}/")
        raise FileNotFoundError(f"Features zip file not found: {zip_file_path}")
    
    print(f"✓ Found zip file: {zip_file_path}")
    
    # Extract zip file if directory is empty or doesn't exist
    if not list(backbone_dir.glob('*.npy')):
        print(f"Extracting {backbone} features...")
        with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
            # Extract all files
            zip_ref.extractall(backbone_dir)
            # If files were extracted with subdirectory structure, move them to root
            for root, dirs, files in os.walk(backbone_dir):
                for file in files:
                    if file.endswith('.npy'):
                        src = Path(root) / file
                        dst = backbone_dir / file
                        if src != dst and not dst.exists():
                            src.rename(dst)
            # Clean up empty subdirectories
            for root, dirs, files in os.walk(backbone_dir, topdown=False):
                for dir_name in dirs:
                    dir_path = Path(root) / dir_name
                    try:
                        if not any(dir_path.iterdir()):
                            dir_path.rmdir()
                    except OSError:
                        pass
        print(f"✓ Extracted {backbone} features")
    else:
        print(f"✓ Features already extracted (found {len(list(backbone_dir.glob('*.npy')))} files)")
    
    print(f"✓ Total files for {backbone}: {len(list(backbone_dir.glob('*.npy')))}")

print("\n✓ All features loaded")


## 6. Setup WandB


In [ ]:
import wandb

# Login to WandB
wandb_key = userdata.get('WANDB_API_KEY')
wandb.login(key=wandb_key)

print("✓ WandB configured")


## 7. Train Models


In [ ]:
import subprocess
import sys

# Training configuration
SPLIT = "recordings"
THRESHOLD = 0.6
EPOCHS = 100
BATCH_SIZE = 128

training_results = {}

for backbone in BACKBONES:
    for variant in VARIANTS:
        model_name = f"{variant}_{backbone}"
        print(f"\n{'='*60}")
        print(f"Training: {model_name}")
        print(f"{'='*60}")
        
        # Select appropriate training script
        if variant == "RNN":
            train_script = "scripts/train_rnn_baseline.py"
        else:
            train_script = "train_er.py"
        
        # Build training command
        cmd = [
            sys.executable, train_script,
            "--backbone", backbone,
            "--variant", variant,
            "--split", SPLIT,
            "--threshold", str(THRESHOLD),
            "--epochs", str(EPOCHS),
            "--batch_size", str(BATCH_SIZE),
            "--use_wandb"
        ]
        
        try:
            # Run training
            result = subprocess.run(cmd, check=True, capture_output=True, text=True)
            print(result.stdout)
            training_results[model_name] = "SUCCESS"
            print(f"✓ {model_name} training completed")
        except subprocess.CalledProcessError as e:
            print(f"✗ {model_name} training failed:")
            print(e.stderr)
            training_results[model_name] = "FAILED"

print(f"\n{'='*60}")
print("Training Summary:")
print(f"{'='*60}")
for model, status in training_results.items():
    status_icon = "✓" if status == "SUCCESS" else "✗"
    print(f"{status_icon} {model}: {status}")


## 8. Evaluate Models


In [ ]:
evaluation_results = {}

for backbone in BACKBONES:
    for variant in VARIANTS:
        model_name = f"{variant}_{backbone}"
        print(f"\n{'='*60}")
        print(f"Evaluating: {model_name}")
        print(f"{'='*60}")
        
        # Build evaluation command
        cmd = [
            sys.executable, "core/evaluate.py",
            "--backbone", backbone,
            "--variant", variant,
            "--split", SPLIT,
            "--threshold", str(THRESHOLD)
        ]
        
        try:
            # Run evaluation
            result = subprocess.run(cmd, check=True, capture_output=True, text=True)
            print(result.stdout)
            evaluation_results[model_name] = "SUCCESS"
            print(f"✓ {model_name} evaluation completed")
        except subprocess.CalledProcessError as e:
            print(f"✗ {model_name} evaluation failed:")
            print(e.stderr)
            evaluation_results[model_name] = "FAILED"

print(f"\n{'='*60}")
print("Evaluation Summary:")
print(f"{'='*60}")
for model, status in evaluation_results.items():
    status_icon = "✓" if status == "SUCCESS" else "✗"
    print(f"{status_icon} {model}: {status}")


## 9. Generate Comparison Report


In [ ]:
# Extract metrics from this notebook's outputs
print("Extracting metrics...")
cmd = [sys.executable, "analysis/extract_metrics.py"]
subprocess.run(cmd, check=True)

# Generate comparison tables
print("\nGenerating comparison tables...")
cmd = [sys.executable, "analysis/comparison_tables.py"]
subprocess.run(cmd, check=True)

# Generate visualizations
print("\nGenerating visualizations...")
cmd = [sys.executable, "analysis/comparison_visualizations.py"]
subprocess.run(cmd, check=True)

print("\n✓ All analysis complete!")
print("Check analysis/outputs/ for results")


# Training with New Backbones: EgoVLP & PerceptionEncoder

Train MLP, Transformer, and RNN models with new video backbones.

## Workflow:
1. Clone repository and install dependencies
2. **Load pre-extracted features from Google Drive**
3. Load annotations
4. Train models with EgoVLP and PerceptionEncoder backbones
5. Evaluate and compare results

## Prerequisites:
- Set Colab secrets: `WANDB_API_KEY`
- **Upload feature zip files** (e.g., `egovlp.zip`, `perceptionencoder.zip`) to Google Drive at `MyDrive/AML_mistake_detection/features/`

## 1. Configuration

In [ ]:
# Configuration
HF_DATASET_REPO = "aexomir/captaincook4d-features" 
REPO_URL = "https://github.com/aexomir/AML_mistake_detection.git"
REPO_BRANCH = "feat/extend-baseline"

# Backbones to train
BACKBONES = ["egovlp", "perceptionencoder"]
# Model variants
VARIANTS = ["MLP", "Transformer", "RNN"]

print(f"Will train {len(BACKBONES)} backbones × {len(VARIANTS)} variants = {len(BACKBONES) * len(VARIANTS)} models")
print(f"Backbones: {BACKBONES}")
print(f"Variants: {VARIANTS}")

## 2. Clone Repository and Install Dependencies

In [ ]:
%%bash -s "$REPO_URL" "$REPO_BRANCH"
cd /content
if [ ! -d "AML_mistake_detection" ]; then
    git clone --branch "$2" "$1" AML_mistake_detection
else
    cd AML_mistake_detection && git pull
fi

In [ ]:
import os
os.chdir('/content/AML_mistake_detection')

%pip install -q -r requirements.txt
%pip install -q wandb huggingface_hub

print("✓ Dependencies installed")

## 3. Load Features from Google Drive

In [ ]:
import zipfile
import os
from pathlib import Path

# Google Drive path for features zip files
FEATURES_DRIVE_PATH = '/content/drive/MyDrive/AML_mistake_detection/features'

# Create data/features directory structure
features_base = Path('/content/AML_mistake_detection/data/features')

for backbone in BACKBONES:
    backbone_dir = features_base / backbone
    backbone_dir.mkdir(parents=True, exist_ok=True)
    
    zip_file_name = f'{backbone}.zip'
    zip_file_path = os.path.join(FEATURES_DRIVE_PATH, zip_file_name)
    
    print(f"Loading {backbone} features from Google Drive...")
    
    # Check if zip file exists
    if not os.path.exists(zip_file_path):
        print(f"❌ ERROR: Zip file not found at: {zip_file_path}")
        print(f"\nPlease ensure {zip_file_name} is uploaded to Google Drive at:")
        print(f"  {FEATURES_DRIVE_PATH}/")
        raise FileNotFoundError(f"Features zip file not found: {zip_file_path}")
    
    print(f"✓ Found zip file: {zip_file_path}")
    
    # Extract zip file if directory is empty or doesn't exist
    if not list(backbone_dir.glob('*.npy')):
        print(f"Extracting {backbone} features...")
        with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
            # Extract all files
            zip_ref.extractall(backbone_dir)
            # If files were extracted with subdirectory structure, move them to root
            for root, dirs, files in os.walk(backbone_dir):
                for file in files:
                    if file.endswith('.npy'):
                        src = Path(root) / file
                        dst = backbone_dir / file
                        if src != dst and not dst.exists():
                            src.rename(dst)
            # Clean up empty subdirectories
            for root, dirs, files in os.walk(backbone_dir, topdown=False):
                for dir_name in dirs:
                    dir_path = Path(root) / dir_name
                    try:
                        if not any(dir_path.iterdir()):
                            dir_path.rmdir()
                    except OSError:
                        pass
        print(f"✓ Extracted {backbone} features")
    else:
        print(f"✓ Features already extracted (found {len(list(backbone_dir.glob('*.npy')))} files)")
    
    print(f"✓ Total files for {backbone}: {len(list(backbone_dir.glob('*.npy')))}")

print("\n✓ All features loaded")

## 4. Setup WandB

In [ ]:
import wandb

# Login to WandB
wandb_key = userdata.get('WANDB_API_KEY')
wandb.login(key=wandb_key)

print("✓ WandB configured")

## 5. Train Models

In [ ]:
import subprocess
import sys

# Training configuration
SPLIT = "recordings"
THRESHOLD = 0.6
EPOCHS = 100
BATCH_SIZE = 128

training_results = {}

for backbone in BACKBONES:
    for variant in VARIANTS:
        model_name = f"{variant}_{backbone}"
        print(f"\n{'='*60}")
        print(f"Training: {model_name}")
        print(f"{'='*60}")
        
        # Select appropriate training script
        if variant == "RNN":
            train_script = "scripts/train_rnn_baseline.py"
        else:
            train_script = "train_er.py"
        
        # Build training command
        cmd = [
            sys.executable, train_script,
            "--backbone", backbone,
            "--variant", variant,
            "--split", SPLIT,
            "--threshold", str(THRESHOLD),
            "--epochs", str(EPOCHS),
            "--batch_size", str(BATCH_SIZE),
            "--use_wandb"
        ]
        
        try:
            # Run training
            result = subprocess.run(cmd, check=True, capture_output=True, text=True)
            print(result.stdout)
            training_results[model_name] = "SUCCESS"
            print(f"✓ {model_name} training completed")
        except subprocess.CalledProcessError as e:
            print(f"✗ {model_name} training failed:")
            print(e.stderr)
            training_results[model_name] = "FAILED"

print(f"\n{'='*60}")
print("Training Summary:")
print(f"{'='*60}")
for model, status in training_results.items():
    status_icon = "✓" if status == "SUCCESS" else "✗"
    print(f"{status_icon} {model}: {status}")

## 6. Evaluate Models

In [ ]:
evaluation_results = {}

for backbone in BACKBONES:
    for variant in VARIANTS:
        model_name = f"{variant}_{backbone}"
        print(f"\n{'='*60}")
        print(f"Evaluating: {model_name}")
        print(f"{'='*60}")
        
        # Build evaluation command
        cmd = [
            sys.executable, "core/evaluate.py",
            "--backbone", backbone,
            "--variant", variant,
            "--split", SPLIT,
            "--threshold", str(THRESHOLD)
        ]
        
        try:
            # Run evaluation
            result = subprocess.run(cmd, check=True, capture_output=True, text=True)
            print(result.stdout)
            evaluation_results[model_name] = "SUCCESS"
            print(f"✓ {model_name} evaluation completed")
        except subprocess.CalledProcessError as e:
            print(f"✗ {model_name} evaluation failed:")
            print(e.stderr)
            evaluation_results[model_name] = "FAILED"

print(f"\n{'='*60}")
print("Evaluation Summary:")
print(f"{'='*60}")
for model, status in evaluation_results.items():
    status_icon = "✓" if status == "SUCCESS" else "✗"
    print(f"{status_icon} {model}: {status}")

## 7. Generate Comparison Report

In [ ]:
# Extract metrics from this notebook's outputs
print("Extracting metrics...")
cmd = [sys.executable, "analysis/extract_metrics.py"]
subprocess.run(cmd, check=True)

# Generate comparison tables
print("\nGenerating comparison tables...")
cmd = [sys.executable, "analysis/comparison_tables.py"]
subprocess.run(cmd, check=True)

# Generate visualizations
print("\nGenerating visualizations...")
cmd = [sys.executable, "analysis/comparison_visualizations.py"]
subprocess.run(cmd, check=True)

print("\n✓ All analysis complete!")
print("Check analysis/outputs/ for results")